# Interacting with Neo4j

## Using py2neo to connect to Neo4j and clear the database

In [ ]:
from py2neo import Graph
graph = Graph("bolt://127.0.0.1:7687")
graph.delete_all()

print("Number of nodes: {} ".format(len(graph.nodes)))
print("Number of relationships: {} ".format(len(graph.nodes)))

## Querying via ipython-cypher 

In [ ]:
%load_ext cypher

### Nodes

In [ ]:
%%cypher
CREATE (person1:Person {name:'Asdrubal'}),
       (person2:Person {name:'Marcos'}   )

### Relationships

In [ ]:
%%cypher
CREATE (person1:Person {name:'Matheus'})-[:LIKES]->(movie1:Movie {title:'Inglourious Basterds', year:2009}),
       (person2:Person {name:'Lais'}   )-[:LIKES]->(movie1)

### Querying

In [ ]:
%%cypher
MATCH (person:Person)-[:LIKES]->(movie:Movie)
RETURN person.name as person_name, movie

In [ ]:
%%cypher
MATCH (person:Person)-[:LIKES]->(movie:Movie)
WHERE  person.name = 'Matheus'
RETURN person.name as person_name, movie.title

## Visualizing the graph

In [ ]:
import sys
sys.path.insert(0, '../../resources/python/')
!mkdir figure

In [ ]:
from datasci4health.graphdrawer.visgraph import draw

options = {}
draw(graph, options, physics=True)

In [ ]:
options = {"Person": "name", "Movie": "title"}
draw(graph, options, physics=True)

## Programmatical querying via py2neo

Examples adapted from http://nicolewhite.github.io/neo4j-jupyter/

In [ ]:
from py2neo import Node, Relationship

### Nodes

In [ ]:
nicole = Node("Person", name="Nicole", age=24)
drew   = Node("Person", name="Drew", age=20)

mtdew    = Node("Drink", name="Mountain Dew", calories=9000)
cokezero = Node("Drink", name="Coke Zero", calories=0)

coke     = Node("Manufacturer", name="Coca Cola")
pepsi    = Node("Manufacturer", name="Pepsi")

graph.create(nicole | drew | mtdew | cokezero | coke | pepsi)

In [ ]:
options = {"Person": "name", "Drink": "name", "Manufacturer": "name", "Movie": "title"}
draw(graph, options, physics=True)

### Relationships

In [ ]:
from py2neo import Relationship

graph.create(Relationship(nicole, "LIKES", cokezero))
graph.create(Relationship(nicole, "LIKES", mtdew))
graph.create(Relationship(drew,   "LIKES", mtdew))
graph.create(Relationship(coke,   "MAKES", cokezero))
graph.create(Relationship(pepsi,  "MAKES", mtdew))

draw(graph, options, physics=True)

### py2neo cypher

In [ ]:
query = """
MATCH (person:Person)-[:LIKES]->(drink:Drink)
RETURN person.name AS name, drink.name AS drink
"""

data = graph.run(query)

for d in data:
    print(d)

## ipython-cypher + pandas

In [ ]:
results = %cypher MATCH (person:Person)-[:LIKES]->(drink:Drink) \
                  RETURN person.name AS name, drink.name AS drink
    
df = results.get_dataframe()

df

## ipython-cypher + networkx

In [ ]:
import networkx as nx
%matplotlib inline

results = %cypher MATCH p = (:Person)-[:LIKES]->(:Drink) RETURN p

g = results.get_graph()

nx.draw(g)

In [ ]:
g.nodes(data=True)

In [ ]:
nx.degree(g)

## jgraph
http://patrickfuller.github.io/jgraph/examples/ipython.html